# Cleaning and organising external data

Using raw csv files of exam results and external indices for each school, this notebook cleans and organises into a readable format for statistical analysis. 

The location of the following required csv files needs to be specified:

* external_outer_indicators.csv
* A2 results 2016-2016.csv

The end output is a csv file that is saved in data/clean called

* school_ext_data.csv



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Inputs
external_dir = r"C:\Users\Andris\Documents\GitHub\data\raw\external_outer_indicators.csv"
exam_dir = r"C:\Users\Andris\Documents\GitHub\data\raw\A2 results 2014-16.csv"

# Outputs
out_dir = r"C:\Users\Andris\Documents\GitHub\data\clean\school_ext_data.csv"


In [18]:
# Reading all indicators
df_tot = pd.read_csv(external_dir)

df_tot.head(5)

URN  IDACI  Effectiveness  Teach_quality  L_M_index PercentageFSM
0  100000    4.0            1.0            1.0        1.0            12
1  100005    NaN            1.0            1.0        1.0          25.8
2  100006    NaN            2.0            2.0        2.0          39.4
3  100007    NaN            2.0            2.0        2.0          78.9
4  100008    5.0            2.0            2.0        2.0          28.3

In [19]:
# Reading exam results
df_exam = pd.read_csv(exam_dir)
#df_exam = df_exam.drop(['FSM','IDACI','L_M_index'],axis=1)
df_exam = df_exam[['URN','Year','Subject','Entries',' A*','A','B',' C',' D',' E','No results']]

df_exam.head(5)

URN  Year                Subject  Entries  A*   A  B  C  D  E No results
0  100001  2016                Biology       13   8   2  3  0  0  0          0
1  100001  2016              Chemistry       12   2   7  3  0  0  0          0
2  100001  2016            Mathematics       53  19  22  8  3  1  0          0
3  100001  2016  Mathematics (Further)       13   7   3  2  1  0  0          0
4  100001  2016                Physics       20   3   9  6  2  0  0          0

In [21]:
# -------------------------------------------------------------
# CLEANING DATA
# -------------------------------------------------------------

# Remove any rows with 'Supp'
df_exam = df_exam[df_exam[' A*'].astype('str') != 'Supp']

# Remove unnecessary subjects
df_exam  =df_exam[(df_exam['Subject'] != 'Mathematics (Further)')]
df_exam  =df_exam[df_exam['Subject'] != 'Mathematics (Statistics)']
df_exam  =df_exam[df_exam['Subject'] != 'Mathematics (Pure)']

# Create new column called exam_score
df_exam['exam_score'] = (df_exam[' A*'].astype('int32')*6 + df_exam['A'].astype('int32')*5 
                         + df_exam['B'].astype('int32')*4 + df_exam[' C'].astype('int32')*3 
                         + df_exam[' D'].astype('int32')*2 + df_exam[' E'].astype('int32')*1)/(df_exam['Entries']*6)

# Combining columns
df_exam['Period'] = df_exam['Year'].astype('str')+ '_'+ df_exam['Subject']

# Grouping and organising
df_new = df_exam.groupby(['URN','Period']).agg({'Entries':'sum','exam_score':'sum'})
df_new = df_new.xs(['Entries','exam_score'],axis=1,drop_level=True)
df_new = df_new.unstack().fillna(0)

# Renaming columns
df_new.columns = df_new.columns.map('|'.join)

# Joining exam data with external indicators
df_all = df_new.join(df_tot.set_index('URN'))

# Saving the csv
df_all.to_csv(out_dir)

# Printing the first 5 rows
df_all.head(5)

Entries|2013_Biology  Entries|2013_Chemistry  \
URN                                                    
100001                  19.0                    25.0   
100003                  24.0                    35.0   
100049                   0.0                     0.0   
100050                  23.0                    17.0   
100051                   6.0                     8.0   

        Entries|2013_Mathematics  Entries|2013_Physics  Entries|2014_Biology  \
URN                                                                            
100001                      53.0                  17.0                  16.0   
100003                      77.0                  41.0                  39.0   
100049                       8.0                   0.0                   0.0   
100050                      19.0                   0.0                  13.0   
100051                       0.0                   0.0                   0.0   

        Entries|2014_Chemistry  Entries|2014_Mathematics  \
URN                                                        
100001                    20.0                      41.0   
100003                    37.0                      88.0   
100049                     0.0                       8.0   
100050                    14.0                      16.0   
100051                     0.0                      10.0   

        Entries|2014_Physics  Entries|2015_Biology  Entries|2015_Chemistry  \
URN                                                                          
100001                  17.0                  15.0                    14.0   
100003                  34.0                  26.0                    45.0   
100049                   0.0                   6.0                     6.0   
100050                   0.0                  23.0                    17.0   
100051                   0.0                   0.0                     0.0   

            ...        exam_score|2015_Physics  exam_score|2016_Biology  \
URN         ...                                                           
100001      ...                       0.780702                 0.897436   
100003      ...                       0.833333                 0.833333   
100049      ...                       0.571429                 0.500000   
100050      ...                       0.000000                 0.560606   
100051      ...                       0.000000                 0.000000   

        exam_score|2016_Chemistry  exam_score|2016_Mathematics  \
URN                                                              
100001                   0.819444                     0.839623   
100003                   0.810185                     0.844749   
100049                   0.380952                     0.515873   
100050                   0.633333                     0.666667   
100051                   0.000000                     0.000000   

        exam_score|2016_Physics  IDACI  Effectiveness  Teach_quality  \
URN                                                                    
100001                 0.775000    NaN            NaN            NaN   
100003                 0.865591    NaN            NaN            NaN   
100049                 0.393939    5.0            3.0            3.0   
100050                 0.000000    5.0            2.0            2.0   
100051                 0.000000    5.0            2.0            2.0   

        L_M_index  PercentageFSM  
URN                               
100001        NaN            NaN  
100003        NaN            NaN  
100049        3.0           40.5  
100050        1.0           23.2  
100051        2.0           39.5  

[5 rows x 37 columns]